### ingest_daily_support_tickets

In [1]:
import os
import pandas as pd
import boto3
from io import StringIO
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import urllib

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# ---------- CONFIG ----------
db_config = {
    "host": "LAPTOP-8J4HGE8R",
    "user": "sa",  # change
    "password": "Cache@123", # change
    "database": "codebasics"
}

S3_BUCKET = "careplus-s3-ingest" 
S3_PREFIX = "support-tickets/raw/"  
DATE_TRACKER_FILE = "date_tracker.txt"

import os

AWS_CONFIG = {
    "aws_access_key_id": os.getenv("AWS_ACCESS_KEY"),
    "aws_secret_access_key": os.getenv("SECRET_KEY"),
    "region_name": os.getenv("REGION")
}

In [4]:
# ---------- UTILITY FUNCTIONS ----------
def get_engine(config):
    params = urllib.parse.quote_plus(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={config['host']};"
        f"DATABASE={config['database']};"
        f"UID={config['user']};"
        f"PWD={config['password']}"
    )
    return create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

def upload_to_s3(df, bucket, key):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    s3 = boto3.client('s3', **AWS_CONFIG)
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    print(f"✅ Uploaded to s3://{bucket}/{key}")

def read_last_date(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return f.read().strip()
    return "2025-06-30"  # Starting point before 1st July

def update_last_date(file_path, new_date):
    with open(file_path, 'w') as f:
        f.write(new_date)

def get_next_date(last_date_str):
    last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
    next_date = last_date + timedelta(days=1)
    return next_date.strftime("%Y-%m-%d")

# ---------- MAIN INGESTION LOGIC ----------
def run_ingestion():
    engine = get_engine(db_config)
    last_date = read_last_date(DATE_TRACKER_FILE)
    next_date = get_next_date(last_date)
    #print(next_date)

    # Query only that day’s data
    query = f"""
        SELECT * FROM support_tickets
        WHERE created_at <= '{next_date} and created_at > {next_date}';
    """
    df = pd.read_sql(query, engine)
    #print(df.shape)
    print(df.head())

    if df.empty:
        print(f"⚠️ No data found for {next_date}. Skipping upload.")
        return

    # Upload to S3
    s3_key = f"{S3_PREFIX}support_tickets_{next_date}.csv"
    upload_to_s3(df, S3_BUCKET, s3_key)

    # Update date tracker
    update_last_date(DATE_TRACKER_FILE, next_date)
    print(f"📅 Updated tracker to {next_date}")

# Run
if __name__ == "__main__":
    run_ingestion()

    ticket_id        created_at       resolved_at   agent priority  \
0  TCK0701000  2025-07-01 00:13              None   Sneha   Medium   
1  TCK0701001  2025-07-01 01:24  2025-07-01 02:47   Kavya    Medum   
2  TCK0701002  2025-07-01 01:49  2025-07-01 19:25   Rohit       Lw   
3  TCK0701003  2025-07-01 02:29  2025-07-02 02:08   Sneha      Low   
4  TCK0701004  2025-07-01 03:03  2025-07-02 11:00  Ananya      Hgh   

  num_interactions         IssUeCat   channel    status agent_feedback  
0                7  Payment Failure      Chat      Open                 
1                8   Account Locked  Web Form  Resolved                 
2                3       Bug Report  Web Form  Resolved                 
3          -999999  Payment Failure     Phone  Resolved                 
4                5       Bug Report      Chat  Resolved                 
✅ Uploaded to s3://careplus-s3-ingest/support-tickets/raw/support_tickets_2025-07-03.csv
📅 Updated tracker to 2025-07-03
